### Import data

In [ ]:
include("importData.jl")
include("Hill.jl")
include("plot.jl")

# import drug concentrations
param_lap_dde = CSV.read(".//figures//Dox//params_dox_DDE.csv")
concentrations = permutedims(Vector(param_lap_dde[8,2:end]));
print(concentrations)

# import G1 and G2 data
pop, g2, g1, g2_0, g1_0 = get_data(joinpath("..", "data", "lap.csv"),
                                   joinpath("..", "data", "lap_pop.csv"))

### Hill model, residual functions and optimization

In [ ]:
# lower bound
low = [50.0, 0.001, 0.2, 0.01, 0.01, 0.001, 0.01, 20.0, 0.1, 0.01, 19.0, 0.01, 0.01, 0.001, 0.01, 0.001, 0.01]
# upper bound
high = [250.0, 0.01, 0.4, 0.04, 0.05, 0.02, 0.04, 35.0, 3.0, 0.04, 23.0, 2.0, 0.4, 0.01, 0.04, 0.04, 0.04]

optimizer_result = optimize_hill(low, high)

### Plot Hill curve for all of the DDE model parameters

In [ ]:
using Plots;
ppp = [127.782, 0.00660381, 0.399771, 0.0290668, 0.0322067, 0.019493, 0.0103518, 32.9621, 2.99971, 0.0100256, 21.3454, 0.0178769, 0.18423, 0.00335835, 0.033172, 0.039048, 0.0250736]
effects = zeros(6, 8)
for i in 1:8
    effects[1, i] = hill(ppp[1:4], concentrations[i])
    effects[2, i] = hill(append!([ppp[1]], ppp[5:7]), concentrations[i])
    effects[3, i] = hill(append!([ppp[1]], ppp[8:10]), concentrations[i])
    effects[4, i] = hill(append!([ppp[1]], ppp[11:13]), concentrations[i])
    effects[5, i] = hill(append!([ppp[1], ppp[14]], [0, ppp[15]]), concentrations[i])
    effects[6, i] = hill(append!([ppp[1], ppp[16]], [0, ppp[17]]), concentrations[i])
end


c = [0.0, 1.0, 10.0, 20.0, 50.0, 125.0, 250.0, 500.0]
plot(c, effects[1, :], label = "alpha", linewidth = 2, xlabel = "concentration [nM]", ylabel = "drug effect")
plot!(c, effects[2, :], label = "beta", linewidth = 2, legend =:right)
plot!(c, effects[5, :], label = "gamma1", linewidth = 2)
plot!(c, effects[6, :], label = "gamma2", linewidth = 2)


In [ ]:
plot(c, effects[3, :], label = "tau1", linewidth = 2, xlabel = "concentration [nM]", ylabel = "drug effect")
plot!(c, effects[4, :], label = "tau2", linewidth = 2, legend =:right)

### Plot the data with the new set of parameters for DDE

In [ ]:
include("DDEmodel.jl")
j = 3 # trial number
data = vcat(g1[:, j]', g2[:, j]')
# for instance we want to plot the data and estimated for some trial 
alpha_ = hill(ppp[1:4], concentrations[j])
beta_ = hill(append!([ppp[1]], ppp[5:7]), concentrations[j])
tau1_ = hill(append!([ppp[1]], ppp[8:10]), concentrations[j])
tau2_ =hill(append!([ppp[1]], ppp[11:13]), concentrations[j])
gamma1_ = hill(append!([ppp[1], ppp[14]], [0, ppp[15]]), concentrations[j])
gamma2_ = hill(append!([ppp[1], ppp[16]], [0, ppp[17]]), concentrations[j])

dde_params = [alpha_, beta_, tau1_, tau2_, gamma1_, gamma2_]
PlotIt(g1_0, g2_0, g1, g2, j, dde_params, data)